# FlyingSquid

In [1]:
#pip install sentence_transformers

In [16]:
from datasets import load_dataset

dataset = load_dataset("glue", "sst2")

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [38]:
train = dataset['train']
test = dataset['test']

records = [
    rb.TextClassificationRecord(
        inputs=text
    )
    for text in train['sentence']
]

records += [
    rb.TextClassificationRecord(
        inputs=text,
        annotation=test.features['label'].names[label]
    )
    for idx, (text, label) in enumerate(zip(test['sentence'], test['label']))
]


rb.log(records, name="weak_supervision_sst2")

  0%|          | 0/69170 [00:00<?, ?it/s]

69170 records logged to http://localhost:6900/ws/rubrix/weak_supervision_sst2


BulkResponse(dataset='weak_supervision_sst2', processed=69170, failed=0)

In [5]:
from rubrix.labeling.text_classification import Rule, WeakLabels
import rubrix as rb

positive_keywords = [
    "funny", "comedy", "love",
    "fun", "entertaining", "romantic",
    "compelling", "worth", "sweet",
    "fascinating", "laughs", "comic",
    "enjoyable", "clever", "perfect",
    "beautiful", "amusing", "powerful",
    ""
]

negative_keywords = [
    "bad", "dull", "worst", ""
]

#  rules defined as Elasticsearch queries
check_out = Rule(query="check out", label="SPAM")
plz = Rule(query="plz OR please", label="SPAM")
subscribe = Rule(query="subscribe", label="SPAM")
my = Rule(query="my", label="SPAM")
song = Rule(query="song", label="HAM")
love = Rule(query="love", label="HAM")

import re

# rules defined as Python labeling functions
def contains_http(record: rb.TextClassificationRecord):
    if "http" in record.inputs["text"]:
        return "SPAM"

def short_comment(record: rb.TextClassificationRecord):
    return "HAM" if len(record.inputs["text"].split()) < 5 else None

def regex_check_out(record: rb.TextClassificationRecord):
    return "SPAM" if re.search(r"check.*out", record.inputs["text"], flags=re.I) else None

from rubrix.labeling.text_classification import load_rules

# bundle our rules in a list
rules = [check_out, plz, subscribe, my, song, love, contains_http, short_comment, regex_check_out]

# optionally add the rules defined in the web app UI
rules += load_rules(dataset="weak_supervision_yt")

# apply the rules to a dataset to obtain the weak labels
weak_labels = WeakLabels(
    rules=rules,
    dataset="weak_supervision_yt"
)

from rubrix.labeling.text_classification import FlyingSquid

# we pass our WeakLabels instance to our FlyingSquid label model
flyingsquid_model = FlyingSquid(weak_labels)

# we fit the model
flyingsquid_model.fit()

flyingsquid_model.score(tie_break_policy="random")
# {'accuracy': 0.832, ...}

Preparing rules:   0%|          | 0/9 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/1711 [00:00<?, ?it/s]

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(


{'HAM': {'precision': 0.9180327868852459,
  'recall': 0.8888888888888888,
  'f1-score': 0.9032258064516128,
  'support': 63},
 'SPAM': {'precision': 0.890625,
  'recall': 0.9193548387096774,
  'f1-score': 0.9047619047619047,
  'support': 62},
 'accuracy': 0.904,
 'macro avg': {'precision': 0.904328893442623,
  'recall': 0.9041218637992832,
  'f1-score': 0.9039938556067587,
  'support': 125},
 'weighted avg': {'precision': 0.9044385245901639,
  'recall': 0.904,
  'f1-score': 0.9039877112135175,
  'support': 125}}

In [4]:
# get your training records with the predictions of the label model
records_for_training = flyingsquid_model.predict()

# log the records to a new dataset in Rubrix
rb.log(records_for_training, name="flyingsquid_results")

# extract training data
training_data = pd.DataFrame(
    [
        {"text": rec.inputs["text"], "label": flyingsquid_model.weak_labels.label2int[rec.prediction[0][0]]}
        for rec in records_for_training
    ]
)

  0%|          | 0/1177 [00:00<?, ?it/s]

1177 records logged to http://localhost:6900/ws/rubrix/flyingsquid_results


In [5]:
# preview training data
training_data

,text,label
0,I really like this song.﻿,0
1,Are those real animals﻿,0
2,"I make guitar covers, please have a look at my...",1
3,"<a href=""https://www.facebook.com/groups/10087...",1
4,i hate rap﻿,0
...,...,...
1172,Hey check out my channel!!!! Please,1
1173,Subscribe﻿,1
1174,Lets party﻿,0
1175,I'd rather hear some propa explicit gangsta ra...,0


In [6]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# fit the classifier
classifier.fit(
    X=training_data.text.tolist(),
    y=training_data.label.values
)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [7]:
# compute the test accuracy
accuracy = classifier.score(
    X=test_df.text.tolist(),
    y=test_df.label.tolist()
)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.864


# Epoxy

In [6]:
from sentence_transformers import SentenceTransformer

class SentenceTransformerModel(object):
    
    def __init__(self, embedding_model_name):
        self.embedding_model = SentenceTransformer(embedding_model_name)
    
    def __call__(self, records):
        texts = [ x.inputs["text"] for x in records ]
        embeddings = self.embedding_model.encode(texts)
        return embeddings

In [7]:
from rubrix.labeling.text_classification import Epoxy
embedding_model_name = "average_word_embeddings_glove.840B.300d"
model = SentenceTransformerModel(embedding_model_name)

embeddings = model(weak_labels.records())

epoxy = Epoxy(weak_labels, embeddings)
epoxy.fit()

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

In [9]:
epoxy._thresholds

[1.0, 1.0, 1.0, 1.0, 1.0, 0.7368421052631579, 1.0, 1.0, 0.894736842105263]

In [11]:
# get your training records with the predictions of the label model
records_for_training = epoxy.predict()

# log the records to a new dataset in Rubrix
rb.log(records_for_training, name="epoxy_results_2")

# extract training data
training_data = pd.DataFrame(
    [
        {"text": rec.inputs["text"], "label": epoxy.weak_labels.label2int[rec.prediction[0][0]]}
        for rec in records_for_training
    ]
)

  0%|          | 0/1194 [00:00<?, ?it/s]

1194 records logged to http://localhost:6900/ws/rubrix/epoxy_results_2


In [12]:
# preview training data
training_data

,text,label
0,I really like this song.﻿,1
1,Are those real animals﻿,0
2,"I make guitar covers, please have a look at my...",1
3,"<a href=""https://www.facebook.com/groups/10087...",1
4,i hate rap﻿,0
...,...,...
1189,Hey check out my channel!!!! Please,1
1190,Subscribe﻿,1
1191,Lets party﻿,0
1192,I'd rather hear some propa explicit gangsta ra...,0


In [13]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# fit the classifier
classifier.fit(
    X=training_data.text.tolist(),
    y=training_data.label.values
)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [14]:
# compute the test accuracy
accuracy = classifier.score(
    X=test_df.text.tolist(),
    y=test_df.label.tolist()
)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.864
